<img src="img/viu_logo.png" width="200">

# 07MAIR - Redes Neuronales y Deep Learning
## Clase 05: Más allá
<img src="img/keras_logo.jpg" width="200">

### Profesores: Adrián Colomer / Gabriel Muñoz
### Autor: Carlos Fernández Musoles / Adrián Colomer Granero


# Temas avanzados
- Functional API
- TensorBoard
- Ir más allá

## Functional API
- Hasta ahora, secuencial
- Suficiente para muchos contextos, limitante para otros complejos
 - inputs independientes, múltiples outputs, ramificaciones internas...

#### Ejemplo X input <-> 1 output: Predicción de precio de ropa de segunda mano

- Inputs: metadata marca, tiempo usado (one hot encoded), foto (imagen), descripción (texto). 
- Modelo con tres submodelos (dense para metadata, RNN para descripción, conv para imagen)

#### Ejemplo 1 input <-> X output: predicción de año de publicación y estilo de un libro

- Un módulo con dos outputs (clasificadores)

- Alternativa a construir modelos separados, pero reutiliza información

In [0]:
!pip install tensorflow==2.0.0-alpha0

     |████████████████████████████████| 79.9MB 118kB/s 
     |████████████████████████████████| 3.0MB 33.8MB/s 
     |████████████████████████████████| 419kB 44.7MB/s 
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [0]:
# ejemplo de modelo construido con la functional API
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

sequential_model = Sequential()
sequential_model.add(layers.Dense(64,activation='relu', input_shape=(64,)))
sequential_model.add(layers.Dense(32,activation='relu'))
sequential_model.add(layers.Dense(1,activation='sigmoid'))

In [0]:
# equivalente en functional API
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras import layers

input_tensor = Input(shape=(64,))
#layer1 = layers.Dense(64,activation='relu')
#x0 = layer1(input_tensor)
x0 = layers.Dense(64,activation='relu')(input_tensor)
#layer2 = layers.Dense(32,activation='relu')
#x1 = layer2(x0)
x1 = layers.Dense(32,activation='relu')(x0)
output_tensor_1 = layers.Dense(1,activation='sigmoid')(x1)
#output_tensor_1 = layer3(x1)
output_tensor_2 = layers.Dense(32,activation='relu')(input_tensor)

functional_model = Model(input_tensor,[output_tensor_1,output_tensor_2] )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
functional_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 64)           4160        input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 32)           2080        dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1)            33          dense_1[0][0]                    
______________________________________________________________________________________________

In [0]:
sequential_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 6,273
Trainable params: 6,273
Non-trainable params: 0
_________________________________________________________________


#### Compilar y entrenar el modelo es igual

### Modelo multi input

- Modelo con dos inputs (atributos y fotos)
- Output: precio de las casas

# House dataset
https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

In [0]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os

# Filtrar las casas de codigos postales poco populares (menos de 10 casas)
MIN_HOUSES_PER_ZIPCODE = 20
# dimensiones de las imagenes (downsampling)
IMAGE_DIM = (32,32)

# cargar los atributos del dataset
def load_house_attributes(inputPath):
        # cargar el dataset con nombres especificados
        cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
        df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)

        # filtrar codigos postales con pocas casas
        zipcodes = df["zipcode"].value_counts().keys().tolist()
        counts = df["zipcode"].value_counts().tolist()

        for (zipcode, count) in zip(zipcodes, counts):
            if count < MIN_HOUSES_PER_ZIPCODE:
                idxs = df[df["zipcode"] == zipcode].index
                df.drop(idxs, inplace=True)

        return df

In [0]:
# procesar los atributos
def process_house_attributes(df, train, test):
        # normalizar (valores 0 a 1) atributos continuos
        continuous = ["bedrooms", "bathrooms", "area"]
        cs = MinMaxScaler()
        trainContinuous = cs.fit_transform(train[continuous])
        testContinuous = cs.transform(test[continuous])

        # one-hot encode el codigo postal
        zipBinarizer = LabelBinarizer().fit(df["zipcode"])
        trainCategorical = zipBinarizer.transform(train["zipcode"])
        testCategorical = zipBinarizer.transform(test["zipcode"])

        # unir todos los atributos y dividir dataset
        trainX = np.hstack([trainCategorical, trainContinuous])
        testX = np.hstack([testCategorical, testContinuous])

        return (trainX, testX)

In [0]:
# cargar imagenes
def load_house_images(df, inputPath):
        images = []

        # cada linea es un data point
        for i in df.index.values:
            # cargar las cuatro imagenes por casa
            basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
            housePaths = sorted(list(glob.glob(basePath)))
            inputImages = []
            outputImage = np.zeros((IMAGE_DIM[0] * 2, IMAGE_DIM[1] * 2, 3), dtype="uint8")

            # por cada imagen, redimensionar 
            for housePath in housePaths:
                # load the input image, resize it to be 32 32, and then
                # update the list of input images
                image = cv2.imread(housePath)
                image = cv2.resize(image, IMAGE_DIM)
                inputImages.append(image)

            # tile the four input images in the output image such the first
            # image goes in the top-right corner, the second image in the
            # top-left corner, the third image in the bottom-right corner,
            # and the final image in the bottom-left corner
            outputImage[0:IMAGE_DIM[0], 0:IMAGE_DIM[1]] = inputImages[0]
            outputImage[0:IMAGE_DIM[0], IMAGE_DIM[1]:IMAGE_DIM[1]*2] = inputImages[1]
            outputImage[IMAGE_DIM[0]:IMAGE_DIM[0]*2, IMAGE_DIM[1]:IMAGE_DIM[1]*2] = inputImages[2]
            outputImage[IMAGE_DIM[0]:IMAGE_DIM[0]*2, 0:IMAGE_DIM[1]] = inputImages[3]

            # add the tiled image to our set of images the network will be
            # trained on
            images.append(outputImage)

        # return our set of images
        return np.array(images) / 255.0

In [0]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers.normalization import BatchNormalization
#from tensorflow.keras.layers.convolutional import Conv2D
#from tensorflow.keras.layers.convolutional import MaxPooling2D
#from tensorflow.keras.layers.core import Activation
#from tensorflow.keras.layers.core import Dropout
#from tensorflow.keras.layers.core import Dense
#from tensorflow.keras.layers import Flatten
#from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

# creacion del modelo MLP (para los atributos numericos)
def create_mlp(dim, regress=False):
        model = Sequential()
        model.add(layers.Dense(8, input_dim=dim, activation="relu"))
        model.add(layers.Dense(4, activation="relu"))

        # check to see if the regression node should be added
        if regress:
            model.add(layers.Dense(1, activation="linear"))

        # return our model
        return model

In [0]:
# crear el modelo CNN
def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
        inputShape = (height, width, depth)
        chanDim = -1

        # define the model input
        inputs = Input(shape=inputShape)

        # crear tantas capas como filtros pasados
        for (i, f) in enumerate(filters):
            if i == 0:
                x = inputs

            # CONV => RELU => BN => POOL
            x = layers.Conv2D(f, (3, 3), padding="same")(x)
            x = layers.Activation("relu")(x)
            x = layers.BatchNormalization(axis=chanDim)(x)
            x = layers.MaxPooling2D(pool_size=(2, 2))(x)
        x = layers.Flatten()(x)
        x = layers.Dense(16)(x)
        x = layers.Activation("relu")(x)
        x = layers.BatchNormalization(axis=chanDim)(x)
        x = layers.Dropout(0.5)(x)

        # output layer
        x = layers.Dense(4)(x)
        x = layers.Activation("relu")(x)

        # check to see if the regression node should be added
        if regress:
            x = layers.Dense(1, activation="linear")(x)

        model = Model(inputs, x)

        return model

In [0]:
# SOLO PARA USO EN GOOGLE COLABORATORY
# para conectar el notebook con la cuenta de gdrive
from google.colab import drive
drive.mount('/content/drive/')
!ls "/content/drive/My Drive/"
BASE_FOLDER = '/content/drive/My Drive/Master IA/Autores/07MAIR/Intro Deep learning/Recursos del profesor/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
 1810_REGISTRO_ACADEMICO_MASTER_INTELIGENCIA_ARTIFICIAL   GRABACIONES_CLAUSTROS
 1904_REGISTRO_ACADEMICO_MASTER_INTELIGENCIA_ARTIFICIAL  'Master IA'
 1910_REGISTRO_ACADEMICO_MASTER_INTELIGENCIA_ARTIFICIAL


In [0]:
# cargar los atributos numericos
print('Cargando atributoss...')
inputPath = BASE_FOLDER+'resources/Houses-dataset/HousesInfo.txt'
df = load_house_attributes(inputPath)

#cargar imagenes
print("[INFO] loading house images...")
images = load_house_images(df, BASE_FOLDER+'resources/Houses-dataset')

Cargando atributoss...
[INFO] loading house images...


In [0]:
images.shape

(384, 64, 64, 3)

In [0]:
from sklearn.model_selection import train_test_split

# dividir imagenes en train y test
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

# normalizar el precio de las casas
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

# procesar atributos numericos
(trainAttrX, testAttrX) = process_house_attributes(df,trainAttrX, testAttrX)

In [0]:
# crear los dos modelos (MLP y CNN)
mlp = create_mlp(trainAttrX.shape[1], regress=False)
cnn = create_cnn(IMAGE_DIM[0] * 2, IMAGE_DIM[1]*2, 3, regress=False)

# unir ambos (concatenarlos)
combinedInput = layers.concatenate([mlp.output, cnn.output])

# add clasificador final tras la concatenacion
x = layers.Dense(4, activation="relu")(combinedInput)
x = layers.Dense(1, activation="linear")(x)

# construir el modelo final
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [0]:
from tensorflow.keras.optimizers import Adam

# utilizamos el optimizador Adam (con learning rate adaptativo)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

# entrenar
print("Entrenar el modelo...")
model.fit([trainAttrX, trainImagesX], trainY,
        validation_data=([testAttrX, testImagesX], testY),
        epochs=50, batch_size=8)

Entrenar el modelo...
Train on 288 samples, validate on 96 samples
Epoch 1/50
288/288 [==============================] - 4s 12ms/sample - loss: 1058.0105 - val_loss: 697.4843
Epoch 2/50
288/288 [==============================] - 3s 9ms/sample - loss: 702.0007 - val_loss: 220.6218
Epoch 3/50
288/288 [==============================] - 3s 9ms/sample - loss: 619.5501 - val_loss: 224.6528
Epoch 4/50
288/288 [==============================] - 3s 9ms/sample - loss: 446.5802 - val_loss: 241.4039
Epoch 5/50
288/288 [==============================] - 3s 9ms/sample - loss: 343.1163 - val_loss: 213.5732
Epoch 6/50
288/288 [==============================] - 3s 9ms/sample - loss: 307.4464 - val_loss: 143.5633
Epoch 7/50
288/288 [==============================] - 3s 9ms/sample - loss: 216.2664 - val_loss: 168.7331
Epoch 8/50
288/288 [==============================] - 3s 9ms/sample - loss: 237.6755 - val_loss: 181.7929
Epoch 9/50
288/288 [==============================] - 3s 9ms/sample - loss: 171.104

In [0]:
# predicciones
print("Predicciones...")
preds = model.predict([testAttrX, testImagesX])

# calcular el error (entre lo predicho y lo esperado)
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# calcular el error medio y su std
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# resultados finales
print("Precio medio: {}. STD: {}".format(df["price"].mean(),df["price"].std()))
print("[INFO] mean: {:.2f}%, std: {:.2f}".format(mean, std))

Predicciones...
Precio medio: 527956.125. STD: 479979.80059985846
[INFO] mean: 29.02%, std: 33.08


#### Ejercicios

- Calcular la misma métrica para el modelo MLP solamente 
 - ¿Obtenemos mejores o peores resultados sin las imágenes?
- Data augmentation para incrementar los data points (¡pocos!)
- Visualizar los filtros y las activaciones máximas del modelo CNN
 - ¿se está utilizando de forma efectiva?

#Tensorboard
Visualizar el proceso de entrenamiento de manera interactiva

In [0]:
%reset
%reload_ext tensorboard.notebook

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


In [0]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
convnet = Sequential()
convnet.add(layers.Conv2D(32,(3,3),input_shape=(32,32,3),activation='relu'))
convnet.add(layers.Conv2D(32,(3,3),activation='relu'))
convnet.add(layers.MaxPooling2D((2,2)))
convnet.add(layers.Conv2D(64,(3,3),activation='relu'))
convnet.add(layers.Conv2D(64,(3,3),activation='relu'))
convnet.add(layers.MaxPooling2D((2,2)))
convnet.add(layers.Flatten())
convnet.add(layers.Dense(512,activation='relu'))
convnet.add(layers.Dense(10,activation='softmax'))

convnet.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [0]:
import datetime, os
logdir = os.path.join("logsClase", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [0]:
%tensorboard --logdir logsClase

Output hidden; open in https://colab.research.google.com to view.

In [0]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = convnet.fit(x_train,y_train,epochs=100,batch_size=64,validation_split=0.2,callbacks=[tensorboard_callback])

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 147s 4ms/sample - loss: 1.4969 - accuracy: 0.4609 - val_loss: 1.4630 - val_accuracy: 0.4853
Epoch 2/100
40000/40000 [==============================] - 147s 4ms/sample - loss: 1.0913 - accuracy: 0.6169 - val_loss: 1.0317 - val_accuracy: 0.6356
Epoch 3/100
40000/40000 [==============================] - 147s 4ms/sample - loss: 0.8743 - accuracy: 0.6941 - val_loss: 0.8521 - val_accuracy: 0.7077
Epoch 4/100
40000/40000 [==============================] - 145s 4ms/sample - loss: 0.7161 - accuracy: 0.7524 - val_loss: 0.8164 - val_accuracy: 0.7179
Epoch 5/100
40000/40000 [==============================] - 145s 4ms/sample - loss: 0.5865 - accuracy: 0.7960 - val_loss: 0.9293 - val_accuracy: 0.7097
Epoch 6/100
40000/40000 [==============================] - 143s 4ms/sample - loss: 0.4710 - accuracy: 0.8356 - val_loss: 0.9342 - val_accuracy: 0.7305
Epoch 7/100
40000/40000 [===================

KeyboardInterrupt: ignored

#### Importante

- Borrar el directorio si se quiere "olvidar" lo entrenado en previas sesiones

# Temas avanzados
- Detección de objetos y segmentación de imagen
  - Arquitecturas enconder-decoder (u-net)->segmentación
  - Single Shot Detector, Faster R-CNN, Mask R-CNN
- Creatividad
 - LSTM para generación de texto
 - GAN (Generative Adversarial Networks)
 - Style transfer
 - Deep Dream
- Compresión - downscaling con autoencoders
- Optimizar
 - optimizar hiperparámetros
 - Ensembles de modelos
- Mantenerse al día
 - arXiV: repositorio de papers (sin revisión!)
 - Kaggle (competiciones, datasets...)
 - https://towardsdatascience.com
 - http://medium.com
 - https://grand-challenge.org/challenges/
 - https://www.pyimagesearch.com/blog/